Serial Schedule Generation Scheme (SGS)

The solver iteratively schedules jobs that are ready (all predecessors completed). For each ready job, it determines the earliest feasible start time by considering:

* Precedence Constraints: Must start after all predecessors finish.
* Resource Constraints: Must start when sufficient resources (renewable capacity and non-renewable stock) are available for its entire duration.

The algorithm maintains a timeline to track resource usage and incrementally finds the first available time slot that satisfies all constraints.

Heuristics Employed:

* Job Prioritization: When multiple jobs are ready, the solver prioritizes jobs based on a calculated "Latest Finish Time" (LFT) heuristic, aiming to schedule activities on the longest path to completion first. This is implemented using a recursive distance calculation from the end of each activity.
* Earliest Feasible Start Time: The solver searches for the absolute earliest time a job can begin by incrementally checking time steps from its earliest possible start time until resource availability is confirmed.
* Mode Selection: If a job has multiple operational modes, the solver defaults to using the first mode listed in the input data.

In [3]:
import json
import time
from typing import Dict, List, Any

def solve_project(json_data: Any) -> Dict:
    start_wall_time = time.perf_counter()

    # Map resources
    res_defs = data.get("resource_definition_mapping", [])
    res_capacities = {r["id"]: r.get("capacity", 0) for r in res_defs}
    res_stocks = {r["id"]: r.get("initial_stock", float('inf')) for r in res_defs}
    res_ids = [r["id"] for r in res_defs]

    # Flatten activities from all jobs into a single global pool
    # We use (project_id, activity_id) as a unique key
    all_activities = {}
    predecessors = {}
    
    for job in data.get("jobs", []):
        p_id = job["project_id"]
        for act in job["activities"]:
            uid = (p_id, act["id"])
            all_activities[uid] = act
            if uid not in predecessors:
                predecessors[uid] = []
                
        # Fill predecessor map using successor lists
        for act in job["activities"]:
            uid = (p_id, act["id"])
            for succ_id in act.get("successors", []):
                succ_uid = (p_id, succ_id)
                if succ_uid not in predecessors:
                    predecessors[succ_uid] = []
                predecessors[succ_uid].append(uid)

    # 2. Priority Rule: Latest Finish Time (LFT) Calculation
    # We perform a backward pass per project to determine priority
    def calculate_lft(job_activities):
        # Simplified: Priority based on longest path to end
        lft_map = {}
        # Identify sink nodes (no successors)
        sinks = [a for a in job_activities if not a.get("successors")]
        
        # Greedy estimate of LFT (Distance from end)
        def get_dist(a_id, memo):
            if a_id in memo: return memo[a_id]
            act_obj = next(a for a in job_activities if a["id"] == a_id)
            if not act_obj.get("successors"):
                memo[a_id] = act_obj["duration"]
                return memo[a_id]
            memo[a_id] = act_obj["duration"] + max(get_dist(s, memo) for s in act_obj["successors"])
            return memo[a_id]

        memo = {}
        for a in job_activities:
            lft_map[a["id"]] = get_dist(a["id"], memo)
        return lft_map

    # Assign priority values (Higher distance = higher priority)
    priorities = {}
    for job in data.get("jobs", []):
        job_lft = calculate_lft(job["activities"])
        for a_id, val in job_lft.items():
            priorities[(job["project_id"], a_id)] = val

    # 3. Scheduling State
    scheduled = {}  # (p_id, a_id): start_time
    renewable_usage = {}  # time: [usage_per_res]
    remaining_uids = set(all_activities.keys())

    # 4. Main Scheduling Loop
    while remaining_uids:
        # Get activities whose predecessors are finished
        eligible = [
            uid for uid in remaining_uids 
            if all(p_uid in scheduled for p_uid in predecessors[uid])
        ]
        
        # Sort by LFT Priority (descending)
        eligible.sort(key=lambda x: priorities[x], reverse=True)
        
        for uid in eligible:
            act = all_activities[uid]
            dur = act["duration"]
            reqs = act.get("resources_required", [0] * len(res_ids))
            cons = act.get("resource_consumption", reqs) # Fallback to reqs if not present

            # A. Determine earliest start based on predecessors
            t_start = 0
            if predecessors[uid]:
                t_start = max(scheduled[p] + all_activities[p]["duration"] for p in predecessors[uid])
            
            # B. Check Resource Constraints (Renewable and Stock)
            feasible = False
            while not feasible:
                # Check Stock (Non-renewable)
                stock_ok = True
                for i, r_id in enumerate(res_ids):
                    if cons[i] > res_stocks[r_id]:
                        stock_ok = False # Constraint violation
                
                if not stock_ok:
                    raise Exception(f"Resource stock exhausted for activity {uid}")

                # Check Capacity (Renewable)
                capacity_ok = True
                for t in range(t_start, t_start + dur):
                    usage_at_t = renewable_usage.get(t, [0] * len(res_ids))
                    for i, r_id in enumerate(res_ids):
                        if usage_at_t[i] + reqs[i] > res_capacities[r_id]:
                            capacity_ok = False
                            break
                    if not capacity_ok: break
                
                if capacity_ok:
                    feasible = True
                else:
                    t_start += 1 # Delay by 1 time unit

            # C. Commit Activity
            scheduled[uid] = t_start
            for t in range(t_start, t_start + dur):
                if t not in renewable_usage:
                    renewable_usage[t] = [0] * len(res_ids)
                for i in range(len(res_ids)):
                    renewable_usage[t][i] += reqs[i]
            
            for i, r_id in enumerate(res_ids):
                res_stocks[r_id] -= cons[i]

            remaining_uids.remove(uid)
            break # Re-evaluate eligibility for next activity

    # 5. Output Formatting
    # Convert keys to strings for JSON compatibility
    final_schedule = {f"P{k[0]}_A{k[1]}": v for k, v in scheduled.items()}
    makespan = max((v + all_activities[k]["duration"] for k, v in scheduled.items()), default=0)
    exec_time = (time.perf_counter() - start_wall_time)*1000

    return {
        "schedule": final_schedule,
        "metrics": {
            "makespan": makespan,
            "total_penalty": 0, 
            "execution_time": f"{exec_time:.4f}s"
        }
    }

In [4]:
with open("C:/Users/Admin/Desktop/mp_j90_a2_nr1.json", 'r') as f:
    data = json.load(f)  

result = solve_project(data)
print(result)

{'schedule': {'P2_A1': 0, 'P2_A4': 0, 'P1_A1': 0, 'P1_A4': 0, 'P2_A6': 7, 'P1_A6': 7, 'P1_A9': 13, 'P2_A9': 13, 'P1_A15': 23, 'P2_A15': 23, 'P2_A2': 0, 'P1_A2': 0, 'P1_A7': 7, 'P2_A7': 7, 'P1_A3': 0, 'P2_A56': 33, 'P2_A3': 0, 'P1_A56': 33, 'P1_A8': 13, 'P1_A5': 8, 'P2_A8': 13, 'P1_A18': 13, 'P2_A5': 8, 'P2_A18': 13, 'P2_A13': 10, 'P1_A13': 10, 'P2_A11': 12, 'P2_A25': 11, 'P1_A25': 11, 'P1_A11': 12, 'P1_A17': 8, 'P2_A17': 8, 'P1_A10': 7, 'P2_A10': 7, 'P1_A19': 7, 'P1_A21': 19, 'P1_A23': 15, 'P2_A19': 7, 'P2_A21': 19, 'P2_A23': 15, 'P1_A28': 8, 'P2_A45': 12, 'P1_A12': 23, 'P1_A16': 7, 'P2_A28': 8, 'P2_A12': 23, 'P1_A45': 12, 'P2_A16': 7, 'P1_A30': 21, 'P2_A30': 21, 'P1_A72': 43, 'P2_A27': 17, 'P2_A72': 43, 'P1_A27': 17, 'P2_A41': 24, 'P1_A14': 15, 'P1_A41': 24, 'P2_A14': 15, 'P1_A31': 20, 'P2_A31': 20, 'P2_A20': 10, 'P1_A20': 10, 'P1_A40': 8, 'P2_A40': 8, 'P2_A32': 8, 'P1_A42': 24, 'P1_A32': 8, 'P2_A42': 24, 'P1_A49': 20, 'P2_A52': 32, 'P2_A38': 29, 'P2_A49': 20, 'P1_A52': 32, 'P1_A38': 